<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Time Series: Forecasting Methods

### Learning Objectives
 - Explore time series forecasting methods.

### Method 1: Naive Approach

Let's look at Walmart's weekly sales data over a two-year period from 2010 to 2012. The data set is separated by store and department, but we'll focus on analyzing one store for simplicity.

#### Read in the data set and take a look at it.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

walmart = pd.read_csv('./datasets/walmart.csv')
walmart.set_index('Date', inplace=True)
walmart.head()

#### Filter the DataFrame to Store 1 sales and aggregate over departments to compute the total sales per store.

In [ ]:
store1 = walmart[walmart.Store == 1]
store1_sales = pd.DataFrame(store1.Weekly_Sales.groupby(store1.index).sum())
store1_sales.reset_index(inplace = True)
store1_sales['Date'] = pd.to_datetime(store1_sales['Date'])
store1_sales.set_index('Date',inplace = True)

In [ ]:
store1_sales.plot();

If we think sales have been stable from the start, we can simply take the last time period's sales and estimate the same value for the next time period. Such a forecasting technique, which assumes that the next expected point is equal to the last observed point, is called a **naive method**.

$$\hat y_{t+1} = y_t$$

We will subset the data to look at the accuracy of each time series forecasting method on the data set. We'll use the first two years (2010–2011) as the "training" data and the last year (2012) as the "testing" data for the purposes of our demonstration.  

In [ ]:
train = store1_sales['2010': '2011']
test = store1_sales['2012']

Let's plot the data to take a look at the subsetting.

In [ ]:
train.Weekly_Sales.plot(figsize=(15,8), title= 'Weekly Sales', fontsize=14)
test.Weekly_Sales.plot(figsize=(15,8), title= 'Weekly Sales', fontsize=14)
plt.show()

#### Let's see how the well the naive method does when forecasting sales.

In [ ]:
dd= np.asarray(train.Weekly_Sales)
y_hat = test.copy()
y_hat['naive'] = dd[len(dd)-1]
plt.figure(figsize=(12,8))
plt.plot(train.index, train['Weekly_Sales'], label='Train')
plt.plot(test.index,test['Weekly_Sales'], label='Test')
plt.plot(y_hat.index,y_hat['naive'], label='Naive Forecast')
plt.legend(loc='best')
plt.title("Naive Forecast")
plt.show()

Let's use RMSE to check the accuracy of our model on the test data set.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms_table = pd.DataFrame()
rms = sqrt(mean_squared_error(test.Weekly_Sales, y_hat.naive))
rms_table.loc['naive','rmse'] = rms
rms_table

We can infer from the RMSE value and the graph above that the naive method isn’t suited for data sets with high variability. It's best suited for stable data sets. We can still improve our score by adopting different techniques, however. Let's try a different technique to attempt to improve the accuracy score.

### Method 2: Simple Average

**Simple average** is a useful method when the values of interest are increasing and decreasing randomly by a small margin but the average remains constant. Simple average forecasts the weekly sales of the next time period to be somewhat similar to the average of all of the previous time periods. 

$$\hat y_{t+1} = \dfrac{1}{x} \sum_{i=1}^{x} y_i$$

In [ ]:
y_hat_avg = test.copy()
y_hat_avg['avg_forecast'] = train['Weekly_Sales'].mean()
plt.figure(figsize=(12,8))
plt.plot(train['Weekly_Sales'], label='Train')
plt.plot(test['Weekly_Sales'], label='Test')
plt.plot(y_hat_avg['avg_forecast'], label='Average Forecast')
plt.legend(loc='best')
plt.show()

Now, let's calculate RMSE to check to accuracy of our model.

In [ ]:
rms = sqrt(mean_squared_error(test.Weekly_Sales, y_hat_avg.avg_forecast))
rms_table.loc['simple_ave','rmse'] = rms
rms_table

This model improved the score a bit. The simple average method works best when the average at each time period remains constant.

### Method 3: Moving Average

When we have data sets in which the sales/value has increased or decreased sharply some time periods ago, simply using the previous average of all of the data (like the simple average method) isn't appropriate. An improvement over the simple average in this case will only take the average of the sales for the last few time periods, as we currently believe that only recent values may matter. This is called the **moving average** technique and it uses a sliding time period window to calculate the average. This model is useful for handling specific or abrupt changes in a system - something going out of stock or a sudden rise in popularity affecting sales.

Using a simple moving average model, we forecast the next value(s) in a time series based on the average of a fixed finite number ($p$) of the previous values. Thus, for all $i > p$:

$$\hat y_{t+1} = \dfrac{1}{p} (y_{i-1} + y_{i-2} + y_{i-3} + ... + y_{i-p})$$

A moving average can be quite effective if you pick the right $p$ for the series.

In [ ]:
# Create copy of dataframe
y_hat_avg = test.copy()

# Get the last value produced by the moving average
y_hat_avg['moving_avg_forecast'] = train['Weekly_Sales'].rolling(8).mean().iloc[-1]

# Plot data over time
plt.figure(figsize=(16,8))
plt.plot(train['Weekly_Sales'], label='Train')
plt.plot(test['Weekly_Sales'], label='Test')
plt.plot(y_hat_avg['moving_avg_forecast'], label='Moving Average Forecast')
plt.legend(loc='best')
plt.show();

We chose the data from the last eight weeks. We will now calculate RMSE to check the accuracy of our model.

In [ ]:
rms = sqrt(mean_squared_error(test.Weekly_Sales, y_hat_avg.moving_avg_forecast))
rms_table.loc['move_ave','rmse'] = rms
rms_table

This didn't perform very well. Perhaps we picked an incorrect $p$, or perhaps this method isn't the best for our data set.

An advancement over the moving average method is the **weighted moving average** method (see example below). In the moving average method, we weigh the past $n$ observations equally. But, we might encounter situations where each of the observations from the past $n$ time-points impacts the forecast in a different way. Typically, weights are chosen so that more recent points matter more. 

$$\hat y_{t+1} = \dfrac{1}{p} (w_1*y_{i-1} + w_2*y_{i-2} + w_3*y_{i-3} + ... + w_p*y_{i-p})$$

In [ ]:
# Example of weighted moving average (using convolution)
x = train['Weekly_Sales']

# Create weights for window
weights = np.array(range(0,8,1))
weights = weights / weights.sum()

from scipy.signal import fftconvolve, convolve # depends on size

plt.figure(figsize=(12,6))
plt.plot(x.values, label='Pre')
plt.plot(fftconvolve(x, weights, mode='valid'), label='Post');

### Method 4: Simple Exponential Smoothing 

Simple average and weighted moving average lie on opposite sides of the spectrum. **Simple exponential smoothing** lies between these two extremes and takes into account all of the data while weighing the data points differently. Simple exponential smoothing will calculate the forecast using weighted averages where the weights decrease exponentially as observations come from further in the past (the closet data points in time are weighted more heavily). 

$$\hat y_{t+1} = \alpha y_t + \alpha (1-\alpha)y_{t-1} + (1-\alpha)^2 y_{t-2} + ...$$

The one-step-ahead forecast for time $t+1$ is a weighted average of all of the observations in the series ($y1,…,yt$). The rate at which the weights decrease is controlled by the parameter, $α$ (which is between 0 and 1).

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
y_hat_avg = test.copy()
fit2 = SimpleExpSmoothing(np.asarray(train['Weekly_Sales'])).fit(smoothing_level=0.6,optimized=False)
y_hat_avg['SES'] = fit2.forecast(len(test))
plt.figure(figsize=(16,8))
plt.plot(train['Weekly_Sales'], label='Train')
plt.plot(test['Weekly_Sales'], label='Test')
plt.plot(y_hat_avg['SES'], label='SES')
plt.legend(loc='best')
plt.show()

Now, let's use RMSE to check to accuracy of our model.

In [ ]:
rms = sqrt(mean_squared_error(test.Weekly_Sales, y_hat_avg.SES))
rms_table.loc['exp_smoothing','rmse'] = rms
rms_table

### Method 5: Holt's Linear Trend

The methods we've looked at so far don't do well when our data have high variations. If our data contain a trend, none of the previous methods would be able to take that into account. A method that can is **Holt's linear trend** method. 

Recall that a time series data set can be decomposed into its trend, seasonality, and residual components. If the data set contains a trend, then Holt's linear trend method can be applied. 

In [ ]:
import statsmodels.api as sm
sm.tsa.seasonal_decompose(train.Weekly_Sales).plot()
result = sm.tsa.stattools.adfuller(train.Weekly_Sales)
plt.show()

From these graphs, we can see that the data set follows an increasing trend, and we can use Holt's linear trend method to forecast the future sales. 

Holt's linear trend is exponential smoothing applied to both the average value of the series (called "level") and trend. 

To generate the forecasting equation, we will add these equations together. 

<h3><center>Level Equation</center></h3> $$\ell_t = \alpha y_t + (1 - \alpha)(\ell_{t-1} + b_{t-1})$$ 
<h3><center>Trend Equation</center></h3> $$b_t = \beta *(\ell_t - \ell_{t-1}) + (1 - \beta)b_{t-1}$$

(Where $t$ is time and $α$, $β$, and $γ$ are the smoothing parameters — each between 0 and 1.)
We will add these equations together to generate the forecast equation. 

<h3><center>Forecast Equation</center></h3>
$$\hat y_{t+h|1}= \ell_t + hb_t $$

We could have also multiplied the trend and level together to generate a multiplicative forecast equation. When the trend is linear, the additive equation is used. When the trend is exponential, the multiplicative equation is used. The multiplicative equation is usually a more stable predictor (but the additive is more intuitive). 

In [ ]:
y_hat_avg = test.copy()

fit1 = Holt(np.asarray(train['Weekly_Sales'])).fit(smoothing_level = 0.3,smoothing_slope = 0.1)
y_hat_avg['Holt_linear'] = fit1.forecast(len(test))

plt.figure(figsize=(16,8))
plt.plot(train['Weekly_Sales'], label='Train')
plt.plot(test['Weekly_Sales'], label='Test')
plt.plot(y_hat_avg['Holt_linear'], label='Holt_linear')
plt.legend(loc='best')
plt.show()

In [ ]:
rms = sqrt(mean_squared_error(test.Weekly_Sales, y_hat_avg.Holt_linear))
rms_table.loc['holt_linear','rmse'] = rms
rms_table

### Method 6: Holt-Winters Method

So far, we haven't taken into account the seasonality of the data set while forecasting. **Holt's Winter** method takes into account both trend and seasonality to forecast future sales. With this method, we will apply exponential smoothing to the seasonal components as well as the level and trend components. 

<h3><center>Level Equation</center></h3> $$L_t = \alpha (y_t - S_{t-s}) + (1 - \alpha)(L_{t-1} + b_{t-1})$$ 
<h3><center>Trend Equation</center></h3> $$b_t = \beta *(L_t - L_{t-1}) + (1 - \beta)b_{t-1}$$
<h3><center>Seasonality Equation</center></h3> $$S_t = \gamma(y_t-L_t)+(1-\gamma)S_{t-s}$$
<h3><center>Forecast Equation</center></h3> $$F_{t+k} = L_t + kb_t+S_{t+k-s}$$

(Where $t$ is time and $α$, $β$, and $γ$ are the smoothing parameters — each between 0 and 1 — and $s$ is the length of the seasonal cycle.)

The trend equation is identical to the equation we used for Holt's linear trend method. The level equation is a weighted average between the seasonally adjusted observation and the non-seasonal forecast for time $t$. The seasonal equation is a weighted average between the current seasonal index and the seasonal index of the same season $s$ time periods ago. 

Just like in Holt's linear trend method, we can use either additive or multiplicative forecasting equations. When the seasonal variations are roughly constant throughout the series, we will use the additive method. When the seasonal variations change depending on the level of the series, we will use the multiplicative method. 

In [ ]:
y_hat_avg = test.copy()
fit1 = ExponentialSmoothing(np.asarray(train['Weekly_Sales']) ,seasonal_periods=7 ,trend='add', seasonal='add',).fit()
y_hat_avg['Holt_Winter'] = fit1.forecast(len(test))
plt.figure(figsize=(16,8))
plt.plot( train['Weekly_Sales'], label='Train')
plt.plot(test['Weekly_Sales'], label='Test')
plt.plot(y_hat_avg['Holt_Winter'], label='Holt_Winter')
plt.legend(loc='best')
plt.show()

Let's look at the RMSE to check accuracy.

In [ ]:
rms = sqrt(mean_squared_error(test.Weekly_Sales, y_hat_avg.Holt_Winter))
rms_table.loc['holt_winter','rmse'] = rms
rms_table

### Method 7: ARIMA

A very popular forecasting method is **ARIMA**, which stands for **autoregressive integrated moving average.** With the exponential smoothing methods we've been using, we've been basing our forecasts on the trend and seasonality in the data. ARIMA models describe the correlations in the data with each other. 

The **seasonal ARIMA** takes into account seasonality (like Holt's Winter method did).

After a time series has been stationarized by differencing, the next step in fitting an ARIMA model is to determine whether AR or MA terms are needed to correct any autocorrelation that remains in the differenced series. By looking at the autocorrelation function (ACF) and partial autocorrelation (PACF) plots of the differenced series, you can tentatively identify the numbers of AR and/or MA terms that are needed. 

[More info](https://people.duke.edu/~rnau/411arim3.htm)

#### Using the ACF and PACF Plots to Pick a Model

Recall from the section on autocorrelation that the autocorrelation function (ACF) is a plot of total correlation between different lag functions. 

In a moving average series of lag $n$, we will not get any correlation between $x(t)$ and $x(t – n -1)$, so the total correlation chart cuts off at $n$th lag. In other words, we can determine the lag for an **MA series** by when the *ACF drops off sharply*. For an **AR series**, this correlation *decreases gradually* without any cut-off value.

If the ACF tells us it is an AR series, then we turn to the PACF. If we find out the partial correlation of each lag, it will cut off after the degree of the AR series. For instance, if we have a AR(1) series, if we exclude the effect of first lag ($x (t-1)$), our second lag ($x (t-2)$) is independent of $x(t)$. Hence, the partial correlation function (PACF) will drop sharply after the first lag.

#### ARIMA Model

In [ ]:
y_hat_avg = test.copy()
fit1 = sm.tsa.statespace.SARIMAX(train.Weekly_Sales, order=(2, 1, 4),seasonal_order=(0,1,1,7)).fit()
y_hat_avg['SARIMA'] = fit1.predict(start="2012-01-06", end="2012-10-26", dynamic=True)
plt.figure(figsize=(16,8))
plt.plot(train['Weekly_Sales'], label='Train')
plt.plot(test['Weekly_Sales'], label='Test')
plt.plot(y_hat_avg['SARIMA'], label='SARIMA')
plt.legend(loc='best')
plt.show()

Let's check the RMSE for accuracy of the model.

In [ ]:
rms = sqrt(mean_squared_error(test.Weekly_Sales, y_hat_avg.SARIMA))
rms_table.loc['sarima','rmse'] = rms
rms_table.sort_values(by='rmse', ascending=False).plot(kind='bar');